In [1]:
import _base_path
import pickle
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from resources.data_io import load_mappings
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from typing import Union, Iterable

In [2]:
DATA    = 'incidents'
MODELS  = [
    'bow-rnd',
    'bow-sup',
    'bow-lr',
    'bow-svm',
    'tf-idf-lr',
    'tf-idf-svm',
    'roberta-base',
    'xlm-roberta-base'
]
LABEL   = 'hazard_category'

# Load Class-Mappings:

In [3]:
class_map = load_mappings(f"../data/{DATA}/splits", LABEL)
class_map

array(['allergens', 'biological', 'chemical',
       'food additives and flavourings', 'food contact materials',
       'foreign bodies', 'fraud', 'migration', 'organoleptic aspects',
       'other hazard', 'packaging defect'], dtype='<U30')

In [4]:
with open('data/incidents/support_zones.json', 'r') as file:
    high_support, low_support = json.load(file)[LABEL]

In [5]:
high_support

['biological']

In [6]:
low_support

['foreign bodies',
 'chemical',
 'fraud',
 'other hazard',
 'packaging defect',
 'organoleptic aspects',
 'food additives and flavourings',
 'migration',
 'food contact materials']

In [7]:
counts = pd.read_csv('data/incidents/incidents_final.csv')[LABEL].value_counts()

class_map = list(zip(
    class_map,
    range(len(class_map)),
    [counts[c] if c in counts else 0 for c in class_map]
))
class_map.sort(key=lambda row:row[2], reverse=True)
class_map

[('biological', 1, 2579),
 ('allergens', 0, 2553),
 ('foreign bodies', 5, 946),
 ('chemical', 2, 584),
 ('fraud', 6, 538),
 ('other hazard', 9, 189),
 ('packaging defect', 10, 101),
 ('organoleptic aspects', 8, 81),
 ('food additives and flavourings', 3, 32),
 ('migration', 7, 14),
 ('food contact materials', 4, 1)]

# Load Results:

In [8]:
results = {}

for m in MODELS:
    r = []
    try:
        for i in range(5):
            with open(f'results/{m}/{m}-{LABEL}-{i:d}.pickle', 'rb') as f:
                r.append(pickle.load(f))
    except FileNotFoundError: continue
    results[m] = r

In [9]:
def calculate_metrics(classes=[c for c, _, _ in class_map]):
    metrics = {}

    for m in results:
        f1        = []
        recall    = []
        precision = []
        accuracy  = []

        idx = [i for c, i, _ in class_map if c in classes]
        for r in results[m]:
            y_true = (r['labels'][:,idx]).astype(int)
            y_pred = (r['predictions'][:,idx] > .5).astype(int)

            f1.append(f1_score(y_true, y_pred, average='macro'))
            recall.append(recall_score(y_true, y_pred, average='macro'))
            precision.append(precision_score(y_true, y_pred, average='macro'))
            #accuracy.append(sum(y_true == y_pred) / len(y_true))
            accuracy.append(accuracy_score(y_true, y_pred))

        metrics[m] = {
            'f1':        (np.mean(f1), np.std(f1)),
            'recall':    (np.mean(recall), np.std(recall)),
            'precision': (np.mean(precision), np.std(precision)),
            'accuracy':  (np.mean(accuracy), np.std(accuracy))
        }
    return metrics

In [ ]:
metrics_all = calculate_metrics()
metrics_high_support = calculate_metrics(high_support)
metrics_low_support = calculate_metrics(low_support)

In [11]:
def print_table(metrics:Iterable[str] = ['f1', 'accuracy']):
    for model in MODELS:
        row =  f'{model.upper()} &\n'

        if model in metrics_all:
            row += ' & '.join([f'${metrics_all[model][metric][0]:.2f} \pm {metrics_all[model][metric][1]:.2f}$' for metric in metrics])
        else:
            row += ' &'*(len(metrics)-1)
            
        row += ' &\n'

        if model in metrics_high_support:
            row += ' & '.join([f'${metrics_high_support[model][metric][0]:.2f} \pm {metrics_high_support[model][metric][1]:.2f}$' for metric in metrics])
        else:
            row += ' &'*(len(metrics)-1)

        row += ' &\n'
        

        if model in metrics_high_support:
            row += ' & '.join([f'${metrics_low_support[model][metric][0]:.2f} \pm {metrics_low_support[model][metric][1]:.2f}$' for metric in metrics])
        else:
            row += ' &'*(len(metrics)-1)
        row += ' \\\\\n'
        print(row)

In [12]:
print_table()

BOW-RND &
$0.13 \pm 0.00$ & $0.00 \pm 0.00$ &
$0.47 \pm 0.01$ & $0.48 \pm 0.01$ &
$0.06 \pm 0.00$ & $0.00 \pm 0.00$ \\

BOW-SUP &
$0.09 \pm 0.00$ & $0.00 \pm 0.00$ &
$0.25 \pm 0.00$ & $0.34 \pm 0.00$ &
$0.00 \pm 0.00$ & $0.67 \pm 0.00$ \\

BOW-LR &
$0.46 \pm 0.02$ & $0.68 \pm 0.01$ &
$0.81 \pm 0.01$ & $0.82 \pm 0.01$ &
$0.38 \pm 0.03$ & $0.76 \pm 0.01$ \\

BOW-SVM &
$0.52 \pm 0.03$ & $0.73 \pm 0.02$ &
$0.85 \pm 0.01$ & $0.86 \pm 0.01$ &
$0.46 \pm 0.04$ & $0.80 \pm 0.01$ \\

TF-IDF-LR &
$0.40 \pm 0.02$ & $0.65 \pm 0.01$ &
$0.78 \pm 0.01$ & $0.79 \pm 0.01$ &
$0.32 \pm 0.02$ & $0.75 \pm 0.01$ \\

TF-IDF-SVM &
$0.47 \pm 0.04$ & $0.70 \pm 0.01$ &
$0.83 \pm 0.01$ & $0.83 \pm 0.01$ &
$0.39 \pm 0.05$ & $0.78 \pm 0.01$ \\

ROBERTA-BASE &
$0.47 \pm 0.02$ & $0.73 \pm 0.04$ &
$0.87 \pm 0.03$ & $0.88 \pm 0.03$ &
$0.39 \pm 0.03$ & $0.78 \pm 0.03$ \\

XLM-ROBERTA-BASE &
$0.45 \pm 0.03$ & $0.72 \pm 0.02$ &
$0.89 \pm 0.02$ & $0.90 \pm 0.02$ &
$0.36 \pm 0.04$ & $0.78 \pm 0.02$ \\

